In [25]:
from pyspark import SparkContext
from itertools import combinations

# Перевірка, чи є активний SparkContext, і використання його, якщо він доступний
from pyspark import SparkContext
try:
    sc = SparkContext.getOrCreate()
except ValueError:
    sc = SparkContext("local", "TriangleCount")

# Завантаження ребер з текстового файлу
lines = sc.textFile("/mnt/data/graph.txt")

# Створення RDD(Resilient Distributed Dataset) з ребер
edges = lines.map(lambda line: tuple(sorted(map(int, line.split()))))

# Видалення дублікатів ребер, якщо такі є
edges = edges.distinct()

# Створення RDD з пар (вершина, сусід)
neighbors = edges.flatMap(lambda edge: [(edge[0], edge[1]), (edge[1], edge[0])])

# Групування за ключем для отримання списків суміжності
adjacency_list = neighbors.groupByKey().mapValues(set).collectAsMap()

# Передача списку суміжності для уникнення посилань на RDD всередині трансформацій
adjacency_list_broadcast = sc.broadcast(adjacency_list)

# Пошук трикутників - виправлена логіка для перевірки всіх ребер трикутника
triangles = edges.flatMap(
    lambda edge: [
        (edge[0], edge[1], v) for v in adjacency_list_broadcast.value[edge[0]]
        if v != edge[1] and v in adjacency_list_broadcast.value[edge[1]]
    ]
)

# Підрахунок кількості унікальних трикутників
triangle_count = triangles.map(lambda x: tuple(sorted([x[0], x[1], x[2]]))).distinct().count()

print(f"Number of triangles: {triangle_count}")

# Зупинка SparkContext
sc.stop()


Number of triangles: 318
